# PyODB Performance Tests


In [1]:
import cProfile
from datetime import datetime
from pathlib import Path
from tqdm import tqdm
from time import sleep, time

from pyodb import PyODB, PyODBCache
from test.test_models.complex_models import ComplexBasic, ComplexContainer, ComplexMulti
from test.test_models.primitive_models import PrimitiveBasic, PrimitiveContainer

In [ ]:
def test_insert_performance():
    pyodb = PyODB(2)
    pyodb.add_type(ComplexBasic)
    pyodb.save(ComplexContainer())
    pyodb.add_type(ComplexMulti)

    for _ in tqdm(range(10)):
        pyodb.save_multiple([ComplexBasic() for _ in range(100)])
        pyodb.save_multiple([ComplexContainer() for _ in range(10)])
        pyodb.save_multiple([ComplexMulti() for _ in range(100)])

        pyodb.delete(ComplexBasic).gt(random_number = 0).commit()
        pyodb.remove_type(ComplexContainer)
        pyodb.add_type(ComplexContainer)
    del pyodb


base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_insert_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
cProfile.run("test_insert_performance()", filepath.as_posix())

In [ ]:
def insert_base_data():
    pyodb = PyODB(persistent=True)
    pyodb.add_type(ComplexBasic)
    pyodb.add_type(ComplexContainer)
    pyodb.add_type(ComplexMulti)

    for _ in tqdm(range(10), desc="Inserting base data"):
        pyodb.save_multiple([ComplexBasic() for _ in range(100)])
        pyodb.save_multiple([ComplexContainer() for _ in range(10)])
        pyodb.save_multiple([ComplexMulti() for _ in range(100)])


def test_select_performance():
    pyodb = PyODB()
    for _ in tqdm(range(100), desc="Testing PrimitiveBasic"):
        pyodb.select(PrimitiveBasic).all()
    for _ in tqdm(range(100), desc="Testing PrimitiveContainer"):
        pyodb.select(PrimitiveContainer).all()
    for _ in tqdm(range(100), desc="Testing ComplexBasic"):
        pyodb.select(ComplexBasic).all()
    for _ in tqdm(range(100), desc="Testing ComplexContainer"):
        pyodb.select(ComplexContainer).all()


base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_select_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
insert_base_data()
cProfile.run("test_select_performance()", filepath.as_posix())

In [2]:
cache = PyODBCache()
cache.pyodb.persistent = True
cache.add_cache("test", lambda x: [PrimitiveBasic() for _ in range(x)], PrimitiveBasic, 2)
print(cache.get_data("test", 100))

start = time()
print(cache["test"])
print(time() - start)

sleep(2)
print(cache.get_data("test", 1))
del cache

[PrimitiveBasic: 211, -205.81095785973758, 'pOU0whT>=P&D"mG[6=W3P,xj3)Gd5ooL}&}5Sg', False, PrimitiveBasic: 938, -630.7083447040644, '(N_m4}PJp', True, PrimitiveBasic: -682, None, '_.X=VT=I', False, PrimitiveBasic: 935, -153.1709253569844, 'I>f_AO7K48/Q,(Q°EPz/EjLdy´,;7fF+X´tA3Q{!lbmU ~uNt46Uz°j´TkuZl!v\p&', False, PrimitiveBasic: 229, -341.10745885719695, 'XCs[FRb´sD:Q&Y cRPeQ\m>>*WBF>,8[:EA;w_~n=Nye`G)7>//;°bm3O{k+;G$E<', True, PrimitiveBasic: -988, None, '`L#Gtfts1$cO[$-5Q=Qk("[pK/Z9k|v=°Fvr´ $\+Vk/§ONoS4{§LUIS§*o\bgN.ppN:#g7x5KJ>Gw}ygMHL(XiOZqK];"VTW#v', True, PrimitiveBasic: -589, 879.6848070225527, 'gjoq"9Id28jR(-H}{a5HT4!&|Y~l^C9<RWLPs`y"*H>7#siF7IFB7HR]-°1oGd/!kPN'A&=1~|(*1dqW$tAAeAu[[!QgpF', True, PrimitiveBasic: 454, -190.41721752945207, 'N'AQ'vSwkFun7}Rz5bu^/\ &aOwKv>3b', False, PrimitiveBasic: 151, 837.7099247039572, 'J*3J8O5!Ef"', False, PrimitiveBasic: -603, 519.1526577794174, '&´HkF&7q<,}1vVuK,*SY9)cp4nPU'm,JxT', True, PrimitiveBasic: -914, -875.4853008973093, 'J8NdkTYo{

## PyODB Examples

In [6]:
class MyType:
    some_data: list[str]
    some_number: int | None

    def __init__(self, number: int):
        self.some_data = ["Hello", "World"]
        self.some_number = number
    
    def __repr__(self) -> str:
        return f"MyType: {self.some_number}"

# Create PyODB instance
pyodb = PyODB()

# Add type and save some instances
pyodb.add_type(MyType)
pyodb.save(MyType(1))
pyodb.save_multiple([MyType(2), MyType(3), MyType(4), MyType(5)])

# Need data elsewhere
select = pyodb.select(MyType)
## only get instances where some_number > 2
select.gt(some_number = 2)
## Res now contains the members
res = select.all()
print(res)

# The select can also be done in a one-liner
res = pyodb.select(MyType).gt(some_number = 2).all()
print(res)

# Delete the saved entries
deleted = pyodb.delete(MyType).gt(some_number = 2).commit()
print(f"Deleted {deleted} entries")

# Count remaining entries
count = pyodb.select(MyType).count()
print(f"{count} entries remaining")

# Clear the database keeping the table definitions
pyodb.clear()

# Show and then remove the type definition
print(pyodb.known_types)
pyodb.remove_type(MyType)
print(pyodb.known_types)


[MyType Number: 3, MyType Number: 4, MyType Number: 5]
[MyType Number: 3, MyType Number: 4, MyType Number: 5]
Deleted 3 entries
2 entries remaining
[<class '__main__.MyType'>]
[]
